# Predicting response times with a network of ReLUs - start simple!
In this notebook I want to get an initial experience with neural networks using ReLU processing units for predicting response times. 

The goal is to gain an intuition of how wide and deep a ReLU network needs to be to model an $n$-transaction concurrency structure.

To gain this intuition, I will model very simple concurrancy structures consisting of a small number of transactions. I expect that such simple structures can be modelled with  simple networks. 
## Summary of results
(Jump over this section if you do not want me to spoil the story ;-) .)

- Based on intuition gained while playing with the small networks, I suspect that an  $m$-unit ReLU network has the representational capacity to model an $n=m$-child concurrency structure.
- The network needs to learn not only how the $n$ inputs are connected to the $m \ge n$ ReLUs (expressed by the $n \times m$ $U$ matrix), but also how the ReLUs feed into each other (expressed by a strictly upper diagonal $m \times m$ $w$ matrix). This learning problem can be formulated as an $m$-stage recurrent neural network. Due to the strict upper diagonal nature of $W$ the unfolded network still has only $m$ ReLUs. I called this model the **N-way RNN**.
- On trivially simple concurrency structures like two serial or two concurrent transactions without additive noise (i.e. processing time spent in the parent) the N-way RNN could be trained "perfectly" (driving the cost function to very small values) using mini-batch SGD with momentum. 
- On the slightly more complex structure of two concurrent children followed by a serial one, the N-way RNN had to be oversized by a factor of two ($m=2 n$) to train with a similar level of reduction in the cost function. Oversizing made the cost surface less "rugged".
- On a small but complex structure like the crossed diamond such an extreme level of cost reduction could not be achieved even after oversizing with a factor of 5. My intuition says the N-way RNN has the capacity to represent even such a complex model, but it is difficult to train it. 

### Possible next steps
- Improve training 
    - Discuss if driving the cost function to such an extremely low value is useful:
        - Pro: such a model is likely to generalize better
        - Contra: noise in the training set may anyway prevent such accurete training
        - Pro: we need extreme accuracy in the noise-free case, so that we can claim that all of the test error is due to additive noise in the data (a high level of additive noise in real-life data will prompt us to look for response-time-impacting transaction-features other than the child respone times)
    - Implement gradient clipping
    - Implement supervised pre-training: vary the regularization of the parameters in $W$ during several training phases so that only a few of them can change in a given phase (i.e. we mimique that we first train a network with $n$ inputs and $m=1$, then $m=2$, etc.)
- Scale up: evaluate the N-way RNN on many different concurrency structures
    - Generate FSB graphs programatically
    - Automate the search for acceptable hyper-parameter values (via randomized search)
    - Evaluate the noise-free modelling error on a wide range of concurrency structures
    - Evaluate the effect of additive noise in the data: can we reliably estimate the amount of additive noise?
- Try it on real data! 
    - Compare it to
        - Generalized mean
        - P-norm
[End of sumamry]

## Paper-and-pencil fit
The reason I turned towards ReLU nets is that they implement a piece-wise linear function, exectly like a concurrency structure does in the noise-free case. Now I would like to examine if it is possible to construct with a paper-and-pencil method a ReLU net that fits very simple concurrency structures.
### Some simple examples
Let's start this examination with considering concurrency structures composed using only either of the serial or sequential operators.
#### Sequential transactions
The response time of a transaction with subsequent child transactions is modeled by adding the response times of the children. Since the response times are positive, a ReLU with unit weights can perfectly implement this operation. 

$$\sum\left(x_0, \ldots, x_n\right) = ReLU\left(x_0 + \ldots + x_n\right)$$

For reasons that will shortly become evident, I note here that this operation can also be implemented by 2-input ReLUs, applying them recursively:

$$\sum\left(x_0, \ldots, x_n\right) = ReLU\left( \ldots ReLU\left( ReLU\left( ReLU\left(x_0\right) + x_1\right) + x_2\right) \ldots + x_n \right)$$  

#### Two concurrent transactions
The response time of a transaction with two concurrent child transactions is modeled by taking the greater of the response times of the children, i.e. by a two-input $\max$ operator. The simplest way to implement this with ReLU operators is

$$\max\left(x_0, x_1\right) = x_1 + ReLU\left(x_0 - x_1\right)$$

Since $x_0$ is non-negative, we can write

$$\max\left(x_0, x_1\right) = ReLU\left(x_1 + ReLU\left(x_0 - x_1\right)\right)$$

#### Three concurrent transactions
To model three concurrent child transactions, we need a three-input $\max$ operator. We can obtain that by recursion:

$$
\max\left(x_0, x_1, x_2\right) = \max\left(\max\left(x_0, x_1\right), x_2\right) \\ = x_2 + ReLU\left(x_1 + ReLU\left(x_0 - x_1\right) - x_2\right)\\ = ReLU\left(x_2 + ReLU\left(x_1 + ReLU\left(x_0 - x_1\right) - x_2\right)\right)
$$

#### Structures composed incrementally

Incremental composition means the method used in previous notebooks an introduced in [Modelling the concurrency pattern of RPC calls with a directed acyclic graph](DAG-01.ipynb).

The $\max\left(x_0, x_1\right)$ and $\sum\left(x_0, x_1\right)$ rules can be applied to any ReLU net modeling an $n$-transaction concurrency structure to obtain a net modeling an $n+1$ transaction concurrency structure composed of the previous structure and a new transaction by the application of one of the '`-`' and '`|`' operators. This means that concurrency structures constructed by the successive application of one of the '`-`' and '`|`' operators to a pre-existing structure and a new transaction (*this is how I generated the concurrency structures in previous experiments*) can be modelled with a **linear chain of ReLUs**.

**A properly constructed $n$-unit ReLU net has the representational capacity to model an arbitrary $n$-transaction incrementally constructed concurrency structure**.

Proper construction means that the ReLUs are organized in a linear chain, the $i$th feeding into the $(i+1)$th with unit weight. Each ReLU has a few additional connections to the input variables. In particular, if the ReLU models concurrent sub-structures, then it has one additional connection with weight $+1$ and another with weight $-1$; if it models sequential sub-structures, then it only has one additional connection with weight $+1$. What prevents the direct construction of such a net is that we do not know which input of the network is connected to which ReLU of the chain (and what weight). The learning task is to find the $n \times n$ weight matrix describing these connections. 

#### Structures composed hierarchically
Incrementally composed structures are built by repeatedly combining an existing concurrency structure and a single transaction with the help of the serial or parallel composition operators.
We can generalize this by adding a new concurrency structure instead if a single transction.

If we construct the concurrency structure by the successive application of the operators to multiple pre-existing concurrency structures, then the resulting structure can be modelled with a **tree of ReLUs**. 

Compared to incrementally composed structures, the learning task is a bit more complicated in the hierarchical case. The ReLUs no longer constitute a linear chain but a tree with unknown structure, which needs to be learnt in addition to how the inputs are connected to the units.

To implement this learning task, a naive approach would be to fix the structure of the ReLU network to a (relatively large number of layers, say $n$) with a large number of units in each layer, and learn the weight matrices describing the layer-to-layer connectivity. The large number of layers would be justified by the incremremental case resulting in a chain; the width of the layers would be influenced by the number of inputs to the network.

This naive approach would ignore the observed pattern that both the incremental and hierarchical case could be modelled with the same number of units as the number of child transactions in the pattern, and would result in $\Ordo(n^4)$ parameters to learn. A better method is to *let the network learn its structure*. Let the matix $\mathbf{U}$ describe how the $n$ inputs of the net are connected to the $m$ ReLUs, and use the $m \times m$ [strictly upper triangular](https://en.wikipedia.org/wiki/Triangular_matrix#Strictly_triangular_matrix) matrix $\mathbf{W}$ to describe how the ReLUs are connected to each other. 

The constraint that the $\mathbf{W}$ matrix is strictly upper triangular ensures that there is no loop among the ReLUs. It is equivalent to numbering the ReLUs starting with $1$ and declaring that each ReLU can only feed into ReLUs with a higher number. 

$W$ is applied $m$ times, each time to a new set of ReLU instances. The procedure results in a recurrent neural network (RNN) covering $m$ "time" steps, in each step consuming the same input contained in the row vector $\mathbf{x}^T$:

$$
\mathbf{h}^{(1)} = ReLU\left(\mathbf{x}^T\cdot \mathbf{U}\right) 
$$

$$
\mathbf{h}^{(t+1)} = ReLU\left(\mathbf{x}^T\cdot \mathbf{U} + \mathbf{h}^{(t)} \cdot \mathbf{W}\right) 
$$

$$
t=1, 2, \dots, m-1 
$$

$$
o = \mathbf{h}^{(n)}_m
$$

where $\mathbf{h}^{(t)}$ is the vector of ReLUs in stage $t$ of the unfolded network. The $o$ output of the network is the output of the last ReLU of the last layer of the network. 

Based on the small scale examples seen earlier, I suspect that any concurrency structure could be accurately represented this way if $m \ge n$. Therefore I will call this model the **n-way RNN model**.

This network has only $n \cdot m + \frac{m \cdot (m-1)}{2}$ parameters to learn. At the first glance, the number of ReLUs in the unfolded RNN appears to be $m^2$. However, exploiting the fact that $\mathbf{W}$ is strictly upper diagonal simplifies the computations and reduces the number of ReLUs to $m$: 

$$
h_1 = ReLU\left( \sum_{i=1}^n x_i \cdot U_{1,i}\right) 
$$

$$
h_{(t+1)} = ReLU\left(x_i \cdot U_{t, i} + \sum_{i=1}^t h_i \cdot W_{i,t}\right) 
$$

$$
t=1, 2, \dots, m-1 
$$

$$
o = h_m
$$


### The general case
Note that the structures constructed hierarchically still constitute only a subset of the possible structures. 

A more generic model of concurrency can be constructed by creating a directed acyclic graph (DAG) with exactly one *start node* (without incoming edge) and exactly one *end node* (without outgoing edge). Each edge represents a bluprint for a child transaction; nodes represent fully synchronising barriers (FSB), meaning that before the outgoing transactions start, *all* incoming transactions must complete. I will call this model the **FSB graph**.

An equivalent definition of the start and end nodes is that by navigating the edges along their direction
 * from the start node all other nodes of the graph are reachable, and 
 * the end node is reachable from any other node of the graph

In a yet more general model each synchronization barrier would be associated with a rule expressing a condition when the outgoing transactions can start (e.g. "$n$ of the incoming must complete") and multiple barriers could consider the same incoming transaction.

Can we expect the above RNN to model an arbitrary FSB graph? I do not endeavour now to answer this with a paper-and-pencil method, though I expect the answer to be "yes". Instead I will later try to validate this experimentally on synthetic data.

## Building the n-way RNN model
To avoid dead ReLUs, I will use leaky units. I will use L1 regularization to encourage sparse solutions. 

In [1]:
from collections import defaultdict
from itertools import count
from pprint import pprint
import theano
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from theano import pp
from theano import tensor as T
from theano import function
from theano.ifelse import ifelse
import numpy as np

/home/dini/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [194]:
def createNWayRNN(n, leak=0, oversizing=1):
    """ @param n: number of inputs to the RNN
        @param leak: slope of the ReLUs for the negative domain 
        @param oversizing: ratio of # of ReLUs to # of inputs
    """
    assert n > 0
    m = n * oversizing
    
    class NWayRNN(object):
        
        # every row (selected by 0th coord) is an observation, 
        # fields in a row (1st coord) are child trx resp time in the observation
        X = T.dmatrix('X')  

        # input-to-ReLU connections
        U = theano.shared(np.zeros((n, m)), 'U')
        
        # inter-ReLU connections
        W = theano.shared(np.zeros((m, m)), 'W')
        
        # all the params adjusted during optimisation
        params = [U, W]
        u_shape = (n, m)
        w_shape = (m, m)
        shapes = [u_shape, w_shape]
        
        # every row (selected by 0th coord) is an observation, 
        # fields in a row (1st coord) are input to the n ReLUs
        xU = T.dot(X, U)
        
#         The t=0 instances of the ReLUs
#         h = [T.nnet.relu( xU, alpha=leak )]
# 
#         The t>0 instances of the ReLUs
#         for t in range(1, n):
#             h.append(T.nnet.relu(xU + T.dot(h[-1], W)))

        h = []
        for t in range(0, m):
            # input from the inputs of the RNN
            input2relu = xU[:,t]  
            for i in range(t):
                # input from the previous ReLUs
                input2relu = input2relu + T.dot(h[i], W[i,t] )
            h.append(T.nnet.relu( input2relu, alpha=leak))
            
        # The expected response time from the model:
        RT = h[-1]

        # Actual parent resp time, each observation on a separate row
        y = T.dvector('y')

        # residual:
        epsilon = y - RT.T
        
        # Mean squared error
        MSE = 0.5 * T.pow(epsilon, 2).mean() 

        l1 = T.dscalar('lambda1')  # regularization parameter for Lasso
#         l2 = T.dscalar('lambda2')  # regularization parameter for Ridge

        L1 = T.sum(abs(U)) + T.sum(abs(W))
        #L2 = (U**2.0).sum()  + (W**2.0).sum()
#         trilW = T.tril(W)  # lower triangle of W
#         Wdiag = T.nlinalg.diag(W)
#         L2 = T.sum(Wdiag * Wdiag)
#         L2 = T.sum(trilW**2.0)
        
        # the cost function
        E = MSE + l1*L1 #+ l2*L2
        step_size = T.dscalar('step_size')
        momentum = T.dscalar('momentum')

        # the gradients:
        gradEs = [T.grad(E, param) for param in params]

        velocities = [theano.shared(np.zeros(shape)) 
                          for param, shape in zip(params, shapes)]
                
        velocity_updates = [ (velocity, momentum * velocity - step_size * gradE )
                        for velocity, gradE in zip(velocities, gradEs)]
        
        updates = [ (param, param + velocity)
                        for param, velocity in zip(params, velocities)]
        

        training_result_variables = ('RT', 'epsilon', 'MSE', 'E',)
        train = staticmethod(theano.function(
                  inputs = [X, y, step_size, momentum, l1, ], #l2
                  outputs = [RT, epsilon, MSE, E, ] + velocities,
                  updates = velocity_updates + updates
                    ))
        
        prediction_result_variables = 'RT', 'epsilon', 'MSE'
        predict = staticmethod(theano.function(inputs=[X, y], 
                                               outputs=[RT, epsilon, MSE, xU] + h))

        def __init__(self, step_size, step_size_min, momentum, l_min, l_max, training_steps, data, 
                     batch_size, MSE_limit, rng, num_runs=1):
            assert 0.0 <= momentum < 1.0
            self.step_size, self.momentum, self.training_steps,  self.MSE_limit = \
                 step_size,      momentum,      training_steps,       MSE_limit
            self.l_max, self.l_min, self.step_size_min, = l_max, l_min, step_size_min, 

            if verbose:
                print ("Training the model using the following parameters:"
                       "\nstep_size = {},\tlambda_min = {},\tlambda_max = {},\ttraining_steps = {}" \
                        .format(step_size, l_min, l_max, training_steps))

#             for w, shape in zip(self.params, self.shapes):
#                 w.set_value(rng.uniform(low=-1, high=1, size=shape))  
            lambdas = sorted(np.logspace(l_min, l_max, self.training_steps), 
                             reverse=True)
            step_size_delta = (step_size-step_size_min) / self.training_steps
            
            bestMSE = 1e128
            self.training_results = []
            
            for run in range(num_runs):
                if True:
                    self.U.set_value(rng.uniform(low=-1, high=1, size=self.u_shape) * 1e-4)
                    self.W.set_value(rng.uniform(low=-1, high=1, size=self.w_shape)
                                     * (np.ones(self.w_shape) - np.tri(*self.w_shape))
                                    )
                else:
                    self.U.set_value(np.ones(self.u_shape) * 1e-4 )
                    self.W.set_value( (np.ones(self.w_shape) - np.tri(*self.w_shape)) * 1e-0 )

                for i, l_ in zip(range(self.training_steps), lambdas):
                    train_X_samples, train_RT_samples = data.sample(batch_size)
                    result = self.train(train_X_samples, train_RT_samples, self.step_size, 
                                        self.momentum, lambda1=l_, ) #lambda2=l_*100
                    RT, epsilon, MSE, E, gradU, gradW = result

    #                    i < 10 and not i % 3 or \
#                        i < 100 and not i % 30 or \
                    if MSE < bestMSE or \
                       i < 10 and not i % 3 or \
                       i < 100 and not i % 30 or \
                       i < 1000 and i % 300 == 0 or \
                       i< 10000 and i % 3000 == 0 or \
                       i % 10000 == 0:  #MSE < bestMSE or 
                        print 
                        print ("Batch #{}: lambda={:5.3f}, Cost={:5.3f}, "
                               "MSE={:5.3f}, step_size={:5.3f}\nWeights:"
                                .format(i, float(l_), float(E), float(MSE), self.step_size))
                        for param in self.params:
                            print param.get_value()
#                         print "Velocities:"
#                         print gradU
#                         print gradW
                    else:
                        pass #print '{:5.3f}'.format(float(MSE)),
                    if MSE < bestMSE:
                        bestMSE = MSE
                    if MSE < MSE_limit:
                        break
                    self.step_size -= step_size_delta
                print "Batch #{}: lambda={:5.3f}, Cost={:5.3f}, MSE={:5.3f}\nWeights:"\
                        .format(i, float(l_), float(E), float(MSE))
                for param in self.params:
                    print param.get_value()
#                 print "Velocities:"
#                 print gradU
#                 print gradW

                self.training_results.append(
                    dict(zip(self.training_result_variables, result))
                    )
                self.training_results[-1]['training_steps'] = training_steps
                if verbose:
                    print "MSE = {}".format(self.training_results[-1]['MSE'])

        def __call__(self, data):
            prediction_result = self.predict(data.test_X_samples, data.test_RT_samples)
            return dict(zip(self.prediction_result_variables, prediction_result))

        @classmethod
        def evaluate_real(cls, Sampler, num_children, step_size, l_min, l_max, 
                          training_steps, batch_size, N, MSE_limit, rng, parallelSequentialMix=-1, ):
            """ Determine how the real values of the model parameters impact the error.
            """
            results = list() 
            for i in range(N):
                if verbose:
                    print "i = {}".format(i)
                data = Sampler.random_data(num_children=num_children, parallelSequentialMix=parallelSequentialMix, 
                                           training_set_size=5000, validation_set_size=0, test_set_size=5000)
                model = cls(step_size, l_min, l_max, training_steps, data, batch_size, MSE_limit, rng)
                results.append( (data, model.training_results, model(data)))
            return results    ;
        
    return NWayRNN

### Verifying the forward propagation 

In [6]:
%%time
theano.config.optimizer='None'
theano.config.exception_verbosity='high'
RNN2 = createNWayRNN(2, 0)
RNN3 = createNWayRNN(3, 0)

CPU times: user 2.23 s, sys: 35.7 ms, total: 2.27 s
Wall time: 2.4 s


#### Two serial transactions

In [12]:
RNN2.U.set_value([[0, 1], 
                  [0, 1]])
RNN2.W.set_value([[0, 0], 
                  [0, 0]])
resu= RNN2.predict(np.array([[1 , 2], 
                             [34, 5], 
                             [6 , 6]]), np.array([3, 39, 12]))
print resu[:3]
print resu[3]
print resu[4:]

[array([  3.,  39.,  12.]), array([ 0.,  0.,  0.]), array(0.0)]
[[  0.   3.]
 [  0.  39.]
 [  0.  12.]]
[array([ 0.,  0.,  0.]), array([  3.,  39.,  12.])]


#### Two concurrent transactions

In [11]:
RNN2.U.set_value([[ 1, 0], 
                  [-1, 1]])
RNN2.W.set_value([[0, 1], 
                  [0, 0]])
resu= RNN2.predict(np.array([[1 , 2], 
                             [34, 5], 
                             [6 , 6]]), np.array([2, 34, 6]))
print resu[:3]
print resu[3]
print resu[4:]

[array([  2.,  34.,   6.]), array([ 0.,  0.,  0.]), array(0.0)]
[[ -1.   2.]
 [ 29.   5.]
 [  0.   6.]]
[array([  0.,  29.,   0.]), array([  2.,  34.,   6.])]


#### Three concurrent transactions

In [13]:
RNN3.U.set_value([[ 1,  0, 0], 
                  [-1,  1, 0],
                  [ 0, -1, 1]
                 ])
RNN3.W.set_value([[0, 1, 0], 
                  [0, 0, 1],
                  [0, 0, 0]
                 ])
resu= RNN3.predict(np.array([[1 , 2, 3], 
                             [34, 5, 6], 
                             [6 , 7, 6]]), np.array([3, 34, 7]))
print resu[:3]
print resu[3]
print resu[4:]

[array([  3.,  34.,   7.]), array([ 0.,  0.,  0.]), array(0.0)]
[[ -1.  -1.   3.]
 [ 29.  -1.   6.]
 [ -1.   1.   6.]]
[array([  0.,  29.,   0.]), array([  0.,  28.,   1.]), array([  3.,  34.,   7.])]


#### Two concurrent plus a serial transaction
$o=x_0+\max(x_1, x_2)$

In [14]:
RNN3.U.set_value([[ 0,  0, 1], 
                  [ 0,  1, 0],
                  [ 0, -1, 1]
                 ])
RNN3.W.set_value([[0, 0, 0], 
                  [0, 0, 1],
                  [0, 0, 0]
                 ])
resu= RNN3.predict(np.array([[1 , 2, 3], 
                             [34, 5, 6], 
                             [6 , 7, 6]]), np.array([4, 40, 13]))
print resu[:3]
print resu[3]
print resu[4:]

[array([  4.,  40.,  13.]), array([ 0.,  0.,  0.]), array(0.0)]
[[  0.  -1.   4.]
 [  0.  -1.  40.]
 [  0.   1.  12.]]
[array([ 0.,  0.,  0.]), array([ 0.,  0.,  1.]), array([  4.,  40.,  13.])]


### Absolute value processing units

In [15]:
%%time
#theano.config.optimizer='None'
#theano.config.exception_verbosity='high'
RNN2abs = createNWayRNN(2, -1)
RNN3abs = createNWayRNN(3, -1)

CPU times: user 2.77 s, sys: 117 ms, total: 2.89 s
Wall time: 11.7 s


#### Two serial transactions

In [17]:
RNN2abs.U.set_value([[0, 1], 
                     [0, 1]])
RNN2abs.W.set_value([[0, 0], 
                     [0, 0]])
resu= RNN2abs.predict(np.array([[1 , 2], 
                                [34, 5], 
                                [6 , 6]]), np.array([3, 39, 12]))
print resu[:3]
print resu[3]
print resu[4:]

[array([  3.,  39.,  12.]), array([ 0.,  0.,  0.]), array(0.0)]
[[  0.   3.]
 [  0.  39.]
 [  0.  12.]]
[array([ 0.,  0.,  0.]), array([  3.,  39.,  12.])]


#### Two concurrent transactions

In [20]:
RNN2abs.U.set_value([[ 1, 0.5], 
                     [-1, 0.5]])
RNN2abs.W.set_value([[ 0, 0.5], 
                     [ 0, 0  ]])
resu= RNN2abs.predict(np.array([[1 , 2], 
                                [34, 5], 
                                [6 , 6]]), np.array([2, 34, 6]))
print resu[:3]
print resu[3]
print resu[4:]

[array([  2.,  34.,   6.]), array([ 0.,  0.,  0.]), array(0.0)]
[[ -1.    1.5]
 [ 29.   19.5]
 [  0.    6. ]]
[array([  1.,  29.,   0.]), array([  2.,  34.,   6.])]


#### Three concurrent transactions

In [21]:
RNN3abs.U.set_value([[ 1,  0.5, 0.25], 
                     [-1,  0.5, 0.25],
                     [ 0, -1  , 0.5  ]
                    ])
RNN3abs.W.set_value([[0, 0.5, 0.25], 
                     [0, 0  , 0.5],
                     [0, 0  , 0  ]
                    ])
resu= RNN3abs.predict(np.array([[1 , 2, 3], 
                                [34, 5, 6], 
                                [6 , 7, 6]]), np.array([3, 34, 7]))
print resu[:3]
print resu[3]
print resu[4:]

[array([  3.,  34.,   7.]), array([ 0.,  0.,  0.]), array(0.0)]
[[ -1.    -1.5    2.25]
 [ 29.    13.5   12.75]
 [ -1.     0.5    6.25]]
[array([  1.,  29.,   1.]), array([  1.,  28.,   1.]), array([  3.,  34.,   7.])]


#### Two concurrent plus a serial transaction
$o=x_0+\max(x_1, x_2)$

In [28]:
RNN3abs.U.set_value([[ 0,  0, 1  ], 
                     [ 0,  1, 0.5],
                     [ 0, -1, 0.5]
                    ])
RNN3abs.W.set_value([[0, 0, 0  ], 
                     [0, 0, 0.5],
                     [0, 0, 0  ]
                    ])
resu= RNN3abs.predict(np.array([[1 , 2, 3], 
                                [34, 5, 6], 
                                [6 , 7, 6]]), np.array([4, 40, 13]))
print resu[:3]
print resu[3]
print resu[4:]

[array([  4.,  40.,  13.]), array([ 0.,  0.,  0.]), array(0.0)]
[[  0.   -1.    3.5]
 [  0.   -1.   39.5]
 [  0.    1.   12.5]]
[array([ 0.,  0.,  0.]), array([ 1.,  1.,  1.]), array([  4.,  40.,  13.])]


## Synthetizing data 

So far I used the term concurrency structure with the meaning of "an incremental composition of parametrized transaction blueprints through the `-` (serial execution) and `|` (parallel execution) operators". I also used the term "hierarchically built concurrency structures". All these structures were FSB graphs.

Now I re-define the term "concurrency structure" to mean a generic FSB graph. The terms "incremental concurrency structure" and "hierarchical c.s" keep their meaning.

The problem of synthetizing data can be decomposed into

 1. Collecting samples from an FSB graph
 2. Creating and FSB graph

### Sampling from an FSB graph
The sampling happens in the `collect_events()` methods of the `FSBGraph` and `TrxBp` classes. The parameters these methods take are:

- **events**: a list of (timeStamp, event) tuples; events are represented as strings (this parameter is not used in this notebook)
- **X**: a list of floats, the durations of the child transactions
- **refTime**: the time when the barrier releases the transactions associated with the outgoing edges (i.e. completes the synchronization)

`FSBGraph.collect_events()` walks the edges of the graph in a deterministic topolpgical order and calls `TrxBp.collect_events()` on each of them. 

This latter simulates not only the $x_i$ duration of the $i$th child transaction but also a short $z_i$ time delay in which the parent transaction executes (e.g. processing data related to the child transaction). The values of $z_i$ and $x_i$ are sampled from log-normal distributions:
$$
\log(x_i) \sim \mathcal{N}(location, scale)
$$
$$
\log(z_i) \sim \mathcal{N}(location + \alpha, scale \cdot \beta)
$$

`TrxBp.collect_events()` generates a pair of `start` and `finish` events with timestamps $refTime + z_i$ and $refTime + z_i + x_i$ and appends them to `events`; $x_i$ is also appended to `X`. The method returns the timestamp of the `finish` event. Barriers take the highest of the finish timestamps of the incoming transaction blueprints, and forward that value as `refTime` to the outgoing transaction blueprints.
#### Taking a single sample
The code below implements an FSB graph and generates samples from it.

In [30]:
class Barrier(object):
    def __init__(self, name):
        self.name = name
        self.in_edges = list()
        self.out_edges = list()

    def __str__(self):
        return "Barrier('{}')".format(self.name)
    __repr__ = __str__
    
    class BarrierDict(defaultdict):
        def __missing__(self, key):
            barrier = self[key] = Barrier(key)
            return barrier
                
class TrxBp(object):
    """ A transaction blue-print
    """
    def __init__(self, name, fr, to, loc=0., scale=1., alpha=-1.0, beta=.10):
        fr.out_edges.append(self)
        to.in_edges.append(self)
        self.fr, self.to = fr, to
        self.name, self.loc, self.scale, self.alpha, self.beta = \
             name,      loc,      scale,      alpha,      beta
        
    def __str__(self):
        if self.loc != 0 or self.scale != 1.:
            return "TrxBp('{}', {:3.1f}, {:3.1f})".format(self.name, self.loc, self.scale)
        else:
            return "TrxBp('{}')".format(self.name)
        
    __repr__ = __str__
        
    def collect_events(self, events, X, refTime=0):
        start  = refTime + np.exp(np.random.normal(self.loc + self.alpha, self.scale * self.beta))
        finish = start   + np.exp(np.random.normal(self.loc             , self.scale            ))
        events.extend([(start, "{" + self.name), (finish, self.name + "}")])
        X.append(finish - start)
        return finish
    
class FSBGraph(object):
    
    def __init__(self):
        self.nodes = Barrier.BarrierDict()
        self.edges = list()
        self.node_count = count(0)
        
    def next_node_id(self):
        return "B{}".format(next(self.node_count))
        
    def add_edge(self, from_id, to_id, loc=0., scale=1., alpha=-1.0, beta=.10):
        name = "{}-->{}".format(from_id, to_id)
        self.edges.append(TrxBp(name, self.nodes[from_id], self.nodes[to_id], 
                                loc, scale, alpha, beta)
                         )
        
    def walk_in_topological_order(self, start_node_id, edge_func, node_func):
        """ Visit each edge and non-start node in a deterministic topological order
        """
        pending_edges = { node: set(node.in_edges) for node in self.nodes.values() }
        pending_nodes = [self.nodes[start_node_id]]
        while pending_nodes:
            from_node = pending_nodes.pop(-1)
            for edge in from_node.out_edges:
                to_node = edge.to
                pending_edges[to_node].discard(edge)
                edge_func(edge)
                if not pending_edges[to_node]:
                    pending_nodes.append(to_node)
                    node_func(to_node)
        if any(pending_edges.values()):
            raise ValueError('Cyclic graph!')

    def collect_events(self, start_node_id, end_node_id, events, X, refTime=0):
        finish_times = dict()
        finish_times[self.nodes[start_node_id]] = refTime
        
        def edge_func(trx):
            #print trx
            finish_times[trx] = trx.collect_events(events, X, finish_times[trx.fr])
    
        def node_func(barrier):
            #print barrier
            finish_times[barrier] = max(finish_times[trx] for trx in barrier.in_edges)
            
        self.walk_in_topological_order(start_node_id, edge_func, node_func)
        return finish_times[self.nodes[end_node_id]] - refTime

##### Check the correctness of sampling on two serial transactions

In [111]:
g = FSBGraph()
g.add_edge('start', 'B1', alpha=-100, beta=0.0)
g.add_edge('B1', 'end', alpha=-100, beta=0.0)
X = list()
events = list()
RT = g.collect_events('start', 'end', events, X, refTime=0)
print sum(X) - RT

0.0


##### Check the correctness of sampling on two parallel transactions

In [112]:
g = FSBGraph()
g.add_edge('start', 'end', alpha=-100, beta=0.0)
g.add_edge('start', 'end', alpha=-100, beta=0.0)
X = list()
events = list()
RT = g.collect_events('start', 'end', events, X, refTime=0)
print max(X) - RT

0.0


##### Demonstrate sampling on a crossed diamond structure

Below a 5-edge diamond-like graph is defined:

In [31]:
g = FSBGraph()
g.add_edge('start', 'B1', alpha=-100, beta=0.0)
g.add_edge('start', 'B2', alpha=-100, beta=0.0)
g.add_edge('B1', 'end', alpha=-100, beta=0.0)
g.add_edge('B2', 'end', alpha=-100, beta=0.0)
g.add_edge('B1', 'B2', alpha=-100, beta=0.0)

This is how we can take a sample from it:

In [32]:
X = list()
events = list()
rt = g.collect_events('start', 'end', events, X, refTime=0)

This is the timing of the "begin" and "end" events of the children:

In [33]:
print events

[(3.7200759760208361e-44, '{start-->B1'), (2.7185901167232882, 'start-->B1}'), (3.7200759760208361e-44, '{start-->B2'), (0.81941289902852787, 'start-->B2}'), (2.7185901167232882, '{B1-->end'), (3.7996427345846335, 'B1-->end}'), (2.7185901167232882, '{B1-->B2'), (3.2130600123016571, 'B1-->B2}'), (3.2130600123016571, '{B2-->end'), (4.2428349993464618, 'B2-->end}')]


This is the duration (time span) of the children:

In [34]:
print X

[2.7185901167232882, 0.81941289902852787, 1.0810526178613453, 0.49446989557836885, 1.0297749870448047]


And this is the response time of the parent:

In [35]:
print rt

4.24283499935


#### Sampling en-mass
The below class can generate training, validation and test data en-mass.

In [36]:
class FSBSampler(object):
    
    def __init__(self, graph, num_children, training_set_size=0, validation_set_size=0, test_set_size=0):
        self.graph = graph
        self.num_children = num_children
        if verbose:
            print ("Generating {} training, {} validation and {} test samples "
                   "using the following concurrency structure:" \
                    .format(training_set_size, validation_set_size, test_set_size))
            print graph
        self.train_X_samples     , self.train_RT_samples      = self.sample(training_set_size  ,) 
        self.validation_X_samples, self.validation_RT_samples = self.sample(validation_set_size,)  
        self.test_X_samples      , self.test_RT_samples       = self.sample(test_set_size      ,)

    def sample(self, N, ):
        """ @param N: number of samples to generate
        """
        X_samples = list()
        RT_samples = list()
        for i in range(N):
            events = list()
            X = list()
            RT = self.graph.collect_events(self.graph.start_node, 
                                           self.graph.end_node, 
                                           events, X)
            X_samples.append(X)
            RT_samples.append(RT)
            
        X_samples = np.array(X_samples)
        RT_samples = np.array(RT_samples)
        return X_samples, RT_samples

### How to generate FSB graphs?
By now we have an FSB graph implementation, but we have no algorithm to generate such graphs automatically. We will address this now partially. (Full treatment later on.)
#### Creating FSB graphs manually
Here is some code that helps creating incremental and hierarchical concurrency structures manually.

In [123]:
class _NonSeq(object):
    def __sub__(self, other):
        if isinstance(other, Sequential):
            other.append(self)
            return other
        else:
            return Sequential((self, other))

class _NonPar(object):
    def __or__(self, other):
        if isinstance(other, Parallel):
            other.append(self)
            return other
        else:
            return Parallel((self, other))

class Sequential(list, _NonPar):
    
    def __str__(self):
        return "(" + " - ".join(str(x) for x in self) + ")"
    __repr__ = __str__
        
    def __sub__(self, other):
        if isinstance(other, Sequential):
            other.extend(self)
            return other
        else:
            self.append(other)
            return self
        
    def generate_graph(self, graph, first=None, last=None):
        if first is None: 
            first = graph.next_node_id()
        fr = first
        for structure in self[:-1]:
            to = graph.next_node_id()
            structure.generate_graph(graph, fr, to)
            fr = to
        if last is None:
            last = graph.next_node_id()
        structure.generate_graph(graph, fr, last)
        return first, last
        
class Parallel(list, _NonSeq):

    def __str__(self):
        return "(" + " | ".join(str(x) for x in self) + ")"
    __repr__ = __str__
    
    def __or__(self, other):
        if isinstance(other, Parallel):
            other.extend(self)
            return other
        else:
            self.append(other)
            return self

    def generate_graph(self, graph, first=None, last=None):
        if first is None: 
            first = graph.next_node_id()
        if last is None:
            last = graph.next_node_id()
        for structure in self:
            structure.generate_graph(graph, first, last)
        return first, last
        
class Trx(_NonSeq, _NonPar):
    
    def __init__(self, name, loc=0., scale=1., alpha=-1.0, beta=.10):
        self.name, self.loc, self.scale, self.alpha, self.beta = \
             name,      loc,      scale,      alpha,      beta
        
    def __str__(self):
        if self.loc != 0 or self.scale != 1.:
            return "Trx('{}', {:3.1f}, {:3.1f})".format(self.name, self.loc, self.scale)
        else:
            return "Trx('{}')".format(self.name)
    __repr__ = __str__

    def generate_graph(self, graph, first=None, last=None):
        if first is None: 
            first = graph.next_node_id()
        if last is None:
            last = graph.next_node_id()
        graph.add_edge(first, last, self.loc, self.scale, self.alpha, self.beta)
        return first, last

##### Verify the correctness of FSB graph construction

Two serial transactions:

In [128]:
g = FSBGraph()
structure = Trx('0', alpha=-100, beta=0) - Trx('1', alpha=-100, beta=0)
start_node, end_node = structure.generate_graph(g)
events = list(); X = list()
RT = g.collect_events(start_node, end_node, events, X, refTime=0)
print sum(X[:2]) == RT

True


Two parallel transactions:

In [130]:
g = FSBGraph()
structure = Trx('0', alpha=-100, beta=0)|Trx('1', alpha=-100, beta=0)
start_node, end_node = structure.generate_graph(g)
events = list(); X = list()
RT = g.collect_events(start_node, end_node, events, X, refTime=0)
print max(X[:2]) == RT

True


The below example produces an FSB graph for two concurrent transactions followed by a third one sequentially.

In [135]:
g = FSBGraph()
structure = Trx('0', alpha=-100, beta=0)|Trx('1', alpha=-100, beta=0) - Trx('2', alpha=-100, beta=0)
start_node, end_node = structure.generate_graph(g)
events = list(); X = list()
RT = g.collect_events(start_node, end_node, events, X, refTime=0)
print max(X[:2] + X[2]) == RT

True


## Learn simple noise-free concurrency structures from data
#### Two serial transactions

In [141]:
%%time
verbose = True

# The sole purpose of this class is to act as a name space
class TwoSerial(object):
    graph = FSBGraph()
    
    # alpha=-100, beta=0.0 ensure the child response times do not contain
    # additive noise
    structure = Trx('0', alpha=-100, beta=0.0)-Trx('1', alpha=-100, beta=0.0)
    graph.start_node, graph.end_node = structure.generate_graph(graph)
    data = FSBSampler(graph, num_children=2)
    RNN = createNWayRNN(2, leak=-1.0)
    RNN(step_size=5e-4, step_size_min=3e-5, momentum=0.99, l_min=-2, l_max=-1, 
        training_steps=5000, data=data, 
        batch_size=128, MSE_limit=1e-4, rng=np.random.RandomState(1236))

Generating 0 training, 0 validation and 0 test samples using the following concurrency structure:
Training the model using the following parameters:
step_size = 0.0005,	lambda_min = -2,	lambda_max = -1,	training_steps = 5000

Batch #0: lambda=0.100, Cost=2.331, MSE=2.122, step_size=0.001
Weights:
[[-0.48400233 -0.02670392]
 [ 0.56614351  0.84482333]]
[[ 0.          0.16422491]
 [ 0.          0.        ]]
Velocities:
[[ 0.  0.]
 [ 0.  0.]]
[[ 0.  0.]
 [ 0.  0.]]

Batch #3: lambda=0.100, Cost=2.481, MSE=2.273, step_size=0.000
Weights:
[[-0.48525036 -0.01046996]
 [ 0.56639859  0.85396304]]
[[ 0.          0.17038401]
 [ 0.          0.        ]]
Velocities:
[[-0.00071155  0.0086382 ]
 [ 0.00010141  0.00464437]]
[[ 0.          0.00324189]
 [ 0.          0.        ]]

Batch #6: lambda=0.100, Cost=4.195, MSE=3.984, step_size=0.000
Weights:
[[-0.4894888   0.03526111]
 [ 0.5668717   0.8781322 ]]
[[ 0.          0.18818092]
 [ 0.          0.        ]]
Velocities:
[[-0.00185588  0.01906844]
 [ 0.00

#### Two concurrent transactions

In [144]:
%%time
class TwoConcurrent(object):
    graph = FSBGraph()
    
    # alpha=-100, beta=0.0 ensure the child response times do not contain
    # additive noise
    structure = Trx('0', alpha=-100, beta=0.0)|Trx('1', alpha=-100, beta=0.0)
    graph.start_node, graph.end_node = structure.generate_graph(graph)
    data = FSBSampler(graph, num_children=2)
    RNN = createNWayRNN(2, leak=-1.0)
    RNN(step_size=5e-4, step_size_min=3e-5, momentum=0.99, l_min=-2, l_max=-1, training_steps=5000, data=data, 
        batch_size=100, MSE_limit=1e-4, rng=np.random.RandomState(1236))

Generating 0 training, 0 validation and 0 test samples using the following concurrency structure:
Training the model using the following parameters:
step_size = 0.0005,	lambda_min = -2,	lambda_max = -1,	training_steps = 5000

Batch #0: lambda=0.100, Cost=0.848, MSE=0.639, step_size=0.001
Weights:
[[-0.48400233 -0.02670392]
 [ 0.56614351  0.84482333]]
[[ 0.          0.16422491]
 [ 0.          0.        ]]
Velocities:
[[ 0.  0.]
 [ 0.  0.]]
[[ 0.  0.]
 [ 0.  0.]]

Batch #3: lambda=0.100, Cost=1.847, MSE=1.638, step_size=0.000
Weights:
[[-0.48474855 -0.01891102]
 [ 0.56604198  0.84899566]]
[[ 0.          0.16767864]
 [ 0.          0.        ]]
Velocities:
[[ -4.39658873e-04   4.33841775e-03]
 [ -6.35581854e-05   2.10062628e-03]]
[[ 0.          0.00187484]
 [ 0.          0.        ]]

Batch #6: lambda=0.100, Cost=6.296, MSE=6.088, step_size=0.000
Weights:
[[-0.48816957  0.00985033]
 [ 0.56576662  0.8611338 ]]
[[ 0.          0.18062973]
 [ 0.          0.        ]]
Velocities:
[[-0.00151598 

#### Two concurrent and a serial transaction
This proved to be a difficult task. It became possible only by oversizing the network, i.e. by having more ReLUs than the minimum.

Varying the step size and regularization over a wide range also seems beneficial.

Weight initialization was tried with a) random weights in the [-1, 1] range
b) uniformly $U=10^{-4}$ and $W=1$. The uniform initialization seems to be slightly better. With smaller random weights the situation may be different. Setting $W=0$ or a small value performed badly. Initializing to 0 is bad: the (sub) gradient is 0 in theano.

In [181]:
%%time
class TwoConcurrentOneSerial(object):
    
    graph = FSBGraph()
    
    # alpha=-100, beta=0.0 ensure the child response times do not contain
    # additive noise
    structure = Trx('0', alpha=-100, beta=0.0)|Trx('1', alpha=-100, beta=0.0) \
                    - Trx('1', alpha=-100, beta=0.0)

    graph.start_node, graph.end_node = structure.generate_graph(graph)
    data = FSBSampler(graph, num_children=3)
    RNN = createNWayRNN(3, leak=-1.0, oversizing=2)
    RNN(step_size=1e-5, step_size_min=1e-8, momentum=0.99, l_min=-3, l_max=-1, 
        training_steps=20000, data=data, 
    batch_size=128, MSE_limit=1e-4, rng=np.random.RandomState(1236), num_runs=1)


Generating 0 training, 0 validation and 0 test samples using the following concurrency structure:
Training the model using the following parameters:
step_size = 1e-05,	lambda_min = -3,	lambda_max = -1,	training_steps = 20000

Batch #0: lambda=0.100, Cost=10.598, MSE=9.246, step_size=0.000
Weights:
[[-0.48400233 -0.02670392  0.56614351  0.84482333 -0.56654049  0.16422491]
 [-0.26922836 -0.47590238 -0.18877982 -0.12941572 -0.20713166  0.94789393]
 [ 0.61541508 -0.57833052 -0.58276216 -0.59233817  0.19187812  0.33147493]]
[[ 0.         -0.44674703  0.66078841 -0.05315243  0.18630455 -0.19584781]
 [ 0.          0.          0.36507877 -0.48796341 -0.22655241 -0.88918358]
 [ 0.          0.          0.         -0.13805008  0.64547291 -0.15692515]
 [ 0.          0.          0.          0.          0.72796055  0.31183683]
 [ 0.          0.          0.          0.          0.          0.27239003]
 [ 0.          0.          0.          0.          0.          0.        ]]
Velocities:
[[ 0.  0.  0

##### Documentation of prevoius parametrization attempts for 2+1 

    %%time
    class TwoConcurrentOneSerial(object):

        graph = FSBGraph()

        # alpha=-100, beta=0.0 ensure the child response times do not contain
        # additive noise
        structure = Trx('0', alpha=-100, beta=0.0)|Trx('1', alpha=-100, beta=0.0) \
                        - Trx('1', alpha=-100, beta=0.0)

        graph.start_node, graph.end_node = structure.generate_graph(graph)
        data = FSBSampler(graph, num_children=3)
        RNN = createNWayRNN(3, leak=-1.0)
        RNN(step_size=1e-6, step_size_min=1e-7, momentum=0.99, l_min=-2, l_max=-1, 
            training_steps=100000, data=data, 
            batch_size=128, MSE_limit=1e-4, rng=np.random.RandomState(1236))

    Batch #99999: lambda=0.010, Cost=0.879, MSE=0.784
    Weights:
    [[-0.25208061  0.14176334  0.93676781]
     [ 1.36900841  0.30531112  0.54098996]
     [-0.55285452 -1.14555782 -0.8014165 ]]
    [[ 0.          0.59916667  1.42856104]
     [ 0.          0.         -1.35142314]
     [ 0.          0.          0.        ]]
    Velocities:
    [[  8.22432807e-07  -1.72177094e-07   1.71823598e-07]
     [  2.30487033e-06   1.58551014e-06   1.69631082e-06]
     [  1.37783830e-06   3.22167851e-07   1.34705529e-06]]
    [[  0.00000000e+00   4.41779883e-07   1.35528617e-06]
     [  0.00000000e+00   0.00000000e+00  -3.25706494e-07]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00]]
    MSE = 0.784269825831
    CPU times: user 46min 32s, sys: 3.49 s, total: 46min 36s
    Wall time: 46min 48s

    %%time
    class TwoConcurrentOneSerial(object):

        graph = FSBGraph()

        # alpha=-100, beta=0.0 ensure the child response times do not contain
        # additive noise
        structure = Trx('0', alpha=-100, beta=0.0)|Trx('1', alpha=-100, beta=0.0) \
                        - Trx('1', alpha=-100, beta=0.0)

        graph.start_node, graph.end_node = structure.generate_graph(graph)
        data = FSBSampler(graph, num_children=3)
        RNN = createNWayRNN(3, leak=-1.0, oversizing=2)
        RNN(step_size=1e-5, step_size_min=1e-7, momentum=0.99, l_min=-2, l_max=-1, 
            training_steps=20000, data=data, 
        batch_size=128, MSE_limit=1e-4, rng=np.random.RandomState(1236), num_runs=1)

    Batch #0: lambda=0.100, Cost=11.958, MSE=10.458, step_size=0.000
    Weights:
    [[ 0.0001  0.0001  0.0001  0.0001  0.0001  0.0001]
     [ 0.0001  0.0001  0.0001  0.0001  0.0001  0.0001]
     [ 0.0001  0.0001  0.0001  0.0001  0.0001  0.0001]]
    [[ 0.  1.  1.  1.  1.  1.]
     [ 0.  0.  1.  1.  1.  1.]
     [ 0.  0.  0.  1.  1.  1.]
     [ 0.  0.  0.  0.  1.  1.]
     [ 0.  0.  0.  0.  0.  1.]
     [ 0.  0.  0.  0.  0.  0.]]
    Velocities:
    [[ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]]
    [[ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]]

    Batch #19999: lambda=0.010, Cost=0.095, MSE=0.001
    Weights:
    [[  6.72576482e-01   1.50322925e-01  -4.74762819e-01  -1.80785144e-01
       -4.46388696e-08  -1.67980815e-06]
     [  2.09603647e-01  -5.23328358e-01   3.16062266e-05  -1.23451717e-07
       -1.13927202e-07   1.46165037e-01]
     [  3.41426238e-01  -5.01726647e-01   1.00951363e-04   2.77413819e-07
        2.76089732e-07   5.55095999e-02]]
    [[ 0.          0.31018855  0.42352746  0.45101686  0.44289959  0.65718738]
     [ 0.          0.          0.32850802  0.4326244   0.4278647   0.71255206]
     [ 0.          0.          0.          0.21138273  0.34252603  0.45926553]
     [ 0.          0.          0.          0.          0.20714665  0.42981528]
     [ 0.          0.          0.          0.          0.          0.29139098]
     [ 0.          0.          0.          0.          0.          0.        ]]
    Velocities:
    [[ -7.81629214e-08   1.88735175e-07   2.02169827e-07   1.26652628e-07
        3.20455410e-08  -2.91644737e-08]
     [  1.56650098e-07   1.16582339e-07  -8.04238980e-09   4.34102982e-09
        2.70069350e-09   1.04741239e-07]
     [  1.06758113e-07   4.45677142e-09   2.37505850e-08  -2.12851046e-08
        1.98317429e-08   7.63712259e-08]]
    [[  0.00000000e+00   1.90991621e-08  -2.84434498e-08  -1.04903523e-07
       -1.24133671e-07  -5.55884729e-08]
     [  0.00000000e+00   0.00000000e+00   2.91727255e-08  -8.79954362e-08
       -1.14082675e-07  -2.10948966e-08]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00  -8.69032180e-08
       -1.13433352e-07  -1.88666565e-08]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
       -1.12084522e-07  -1.42374423e-08]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
        0.00000000e+00   2.09951247e-08]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
        0.00000000e+00   0.00000000e+00]]
    MSE = 0.000833945147794
    CPU times: user 9min 53s, sys: 2.98 s, total: 9min 56s
    Wall time: 10min 25s


    %%time
    class TwoConcurrentOneSerial(object):

        graph = FSBGraph()

        # alpha=-100, beta=0.0 ensure the child response times do not contain
        # additive noise
        structure = Trx('0', alpha=-100, beta=0.0)|Trx('1', alpha=-100, beta=0.0) \
                        - Trx('1', alpha=-100, beta=0.0)

        graph.start_node, graph.end_node = structure.generate_graph(graph)
        data = FSBSampler(graph, num_children=3)
        RNN = createNWayRNN(3, leak=-1.0)
        RNN(step_size=1e-5, step_size_min=1e-8, momentum=0.99, l_min=-2, l_max=-1, 
            training_steps=10000, data=data, 
        batch_size=128, MSE_limit=1e-4, rng=np.random.RandomState(1236), num_runs=1)

    Batch #0: lambda=0.100, Cost=10.459, MSE=10.459, step_size=0.000
    Weights:
    [[ 0.0001  0.0001  0.0001]
     [ 0.0001  0.0001  0.0001]
     [ 0.0001  0.0001  0.0001]]
    [[ 0.      0.0001  0.0001]
     [ 0.      0.      0.0001]
     [ 0.      0.      0.    ]]
    Velocities:
    [[ 0.  0.  0.]
     [ 0.  0.  0.]
     [ 0.  0.  0.]]
    [[ 0.  0.  0.]
     [ 0.  0.  0.]
     [ 0.  0.  0.]]

    Batch #4349: lambda=0.037, Cost=0.197, MSE=0.110, step_size=0.000
    Weights:
    [[ -2.31599581e-05   2.92403438e-05   6.44817450e-01]
     [ -3.08492205e-05  -3.07947399e-05   8.62641425e-01]
     [ -3.06532694e-05  -3.06333164e-05   8.72901310e-01]]
    [[  0.00000000e+00  -3.17541776e-05   2.09549564e-05]
     [  0.00000000e+00   0.00000000e+00   8.24240458e-07]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00]]
    Velocities:
    [[  1.90278253e-06   1.40713805e-06   1.23401978e-05]
     [ -1.16565032e-06  -1.16556597e-06   6.70230053e-06]
     [ -1.09450964e-06  -1.09517297e-06  -1.15125409e-05]]
    [[  0.00000000e+00  -9.68629697e-07  -2.27719692e-06]
     [  0.00000000e+00   0.00000000e+00  -3.31669095e-06]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00]]

    Batch #9999: lambda=0.010, Cost=0.196, MSE=0.172
    Weights:
    [[  4.18003804e-08   4.41918301e-07   6.52435644e-01]
     [ -4.74918851e-07  -5.42555202e-07   8.69178461e-01]
     [ -4.54067316e-07  -3.34256868e-07   8.69691934e-01]]
    [[  0.00000000e+00   5.96289449e-07   5.85485141e-07]
     [  0.00000000e+00   0.00000000e+00  -1.21254851e-07]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00]]
    Velocities:
    [[ -1.36530332e-08  -7.63719979e-09  -8.20261778e-08]
     [  4.14945394e-09   3.52205502e-09  -5.60918853e-08]
     [  4.37994792e-09  -1.22122606e-08  -4.73032439e-07]]
    [[  0.00000000e+00  -2.97383194e-09  -4.09741767e-09]
     [  0.00000000e+00   0.00000000e+00   1.43357189e-08]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00]]
    MSE = 0.17203980952
    CPU times: user 4min 42s, sys: 403 ms, total: 4min 42s
    Wall time: 4min 50s

    %%time
    class TwoConcurrentOneSerial(object):

        graph = FSBGraph()

        # alpha=-100, beta=0.0 ensure the child response times do not contain
        # additive noise
        structure = Trx('0', alpha=-100, beta=0.0)|Trx('1', alpha=-100, beta=0.0) \
                        - Trx('1', alpha=-100, beta=0.0)

        graph.start_node, graph.end_node = structure.generate_graph(graph)
        data = FSBSampler(graph, num_children=3)
        RNN = createNWayRNN(3, leak=-1.0)
        RNN(step_size=1e-7, step_size_min=1e-8, momentum=0.99, l_min=-2, l_max=-1, 
            training_steps=10000, data=data, 
            batch_size=128, MSE_limit=1e-4, rng=np.random.RandomState(1236), num_runs=10)

    Batch #9999: lambda=0.010, Cost=  nan, MSE=  nan
    Weights:
    [[ nan  nan  nan]
     [ nan  nan  nan]
     [ nan  nan  nan]]
    [[  0.  nan  nan]
     [  0.   0.  nan]
     [  0.   0.   0.]]
    Velocities:
    [[ nan  nan  nan]
     [ nan  nan  nan]
     [ nan  nan  nan]]
    [[  0.  nan  nan]
     [  0.   0.  nan]
     [  0.   0.   0.]]
    MSE = nan
    CPU times: user 46min 41s, sys: 3.24 s, total: 46min 44s
    Wall time: 47min 6s


    %%time
    class TwoConcurrentOneSerial(object):

        graph = FSBGraph()

        # alpha=-100, beta=0.0 ensure the child response times do not contain
        # additive noise
        structure = Trx('0', alpha=-100, beta=0.0)|Trx('1', alpha=-100, beta=0.0) \
                        - Trx('1', alpha=-100, beta=0.0)

        graph.start_node, graph.end_node = structure.generate_graph(graph)
        data = FSBSampler(graph, num_children=3)
        RNN = createNWayRNN(3, leak=-1.0, oversizing=2)
        RNN(step_size=1e-5, step_size_min=1e-8, momentum=0.99, l_min=-2, l_max=-1, 
            training_steps=20000, data=data, 
        batch_size=128, MSE_limit=1e-4, rng=np.random.RandomState(1236), num_runs=1)
        
    Generating 0 training, 0 validation and 0 test samples using the following concurrency structure:
    <__main__.FSBGraph object at 0x7fac7b65f550>
    Training the model using the following parameters:
    step_size = 1e-05,	lambda_min = -2,	lambda_max = -1,	training_steps = 20000

    Batch #0: lambda=0.100, Cost=15.282, MSE=13.782, step_size=0.000
    Weights:
    [[ 0.0001  0.0001  0.0001  0.0001  0.0001  0.0001]
     [ 0.0001  0.0001  0.0001  0.0001  0.0001  0.0001]
     [ 0.0001  0.0001  0.0001  0.0001  0.0001  0.0001]]
    [[ 0.  1.  1.  1.  1.  1.]
     [ 0.  0.  1.  1.  1.  1.]
     [ 0.  0.  0.  1.  1.  1.]
     [ 0.  0.  0.  0.  1.  1.]
     [ 0.  0.  0.  0.  0.  1.]
     [ 0.  0.  0.  0.  0.  0.]]
    Velocities:
    [[ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]]
    [[ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]]        
     
    Batch #13341: lambda=0.022, Cost=0.191, MSE=0.000
    Weights:
    [[  3.77225724e-01   2.66591333e-01  -3.67739748e-01  -8.95606639e-02
        3.47097390e-06   3.10537705e-05]
     [  1.53339076e-05  -3.99037226e-01  -4.66227680e-01   2.58827276e-06
       -7.41457119e-06   6.53912042e-02]
     [ -8.17557289e-06  -3.91508340e-01  -5.43877944e-01   4.05626188e-06
        1.37232325e-05   3.64278682e-03]]
    [[  0.00000000e+00   3.24480785e-01   2.09313549e-01   3.49943173e-01
        4.18074558e-01   4.39718165e-01]
     [  0.00000000e+00   0.00000000e+00  -3.60910814e-04   5.06139911e-01
        4.67670227e-01   8.10116420e-01]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   3.16761342e-01
        4.02449099e-01   6.00856092e-01]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
        2.37609407e-01   4.88223128e-01]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
        0.00000000e+00   3.37523597e-01]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
        0.00000000e+00   0.00000000e+00]]
    Velocities:
    [[ -1.08786799e-06   4.26814563e-06   3.46983559e-06   9.79562109e-06
        1.23272265e-06  -3.12224799e-07]
     [ -4.93043195e-07   4.81907648e-08  -5.03624399e-07   1.05193863e-06
        1.35135356e-07   1.21855113e-06]
     [  2.14912146e-08  -1.54076104e-06  -1.00913007e-07  -6.26382368e-07
        6.06920893e-07   7.79271092e-07]]
    [[  0.00000000e+00  -2.98052891e-06  -8.84785414e-06  -6.46052864e-06
       -6.83216156e-06  -5.76703523e-06]
     [  0.00000000e+00   0.00000000e+00  -6.09913769e-07  -2.42291645e-06
       -4.43699509e-06   1.33447132e-06]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00  -1.87418228e-06
       -4.11142051e-06   2.29966021e-06]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
       -4.79227942e-06   2.81053200e-07]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
        0.00000000e+00   3.09578841e-06]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
        0.00000000e+00   0.00000000e+00]]
    MSE = 9.89164068125e-05
    CPU times: user 6min 34s, sys: 813 ms, total: 6min 35s
    Wall time: 6min 50s     

    %%time
    class TwoConcurrentOneSerial(object):

        graph = FSBGraph()

        # alpha=-100, beta=0.0 ensure the child response times do not contain
        # additive noise
        structure = Trx('0', alpha=-100, beta=0.0)|Trx('1', alpha=-100, beta=0.0) \
                        - Trx('1', alpha=-100, beta=0.0)

        graph.start_node, graph.end_node = structure.generate_graph(graph)
        data = FSBSampler(graph, num_children=3)
        RNN = createNWayRNN(3, leak=-1.0, oversizing=2)
        RNN(step_size=1e-5, step_size_min=1e-8, momentum=0.99, l_min=-3, l_max=-1, 
            training_steps=20000, data=data, 
        batch_size=128, MSE_limit=1e-4, rng=np.random.RandomState(1236), num_runs=1)

    Batch #0: lambda=0.100, Cost=16.051, MSE=14.551, step_size=0.000
    Weights:
    [[ 0.0001  0.0001  0.0001  0.0001  0.0001  0.0001]
     [ 0.0001  0.0001  0.0001  0.0001  0.0001  0.0001]
     [ 0.0001  0.0001  0.0001  0.0001  0.0001  0.0001]]
    [[ 0.  1.  1.  1.  1.  1.]
     [ 0.  0.  1.  1.  1.  1.]
     [ 0.  0.  0.  1.  1.  1.]
     [ 0.  0.  0.  0.  1.  1.]
     [ 0.  0.  0.  0.  0.  1.]
     [ 0.  0.  0.  0.  0.  0.]]
    Velocities:
    [[ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]]
    [[ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]
     [ 0.  0.  0.  0.  0.  0.]]

    Batch #9633: lambda=0.011, Cost=0.117, MSE=0.000
    Weights:
    [[  4.77657155e-01   1.50682832e-01  -5.17793643e-01  -3.58701822e-01
       -1.48020780e-02  -9.60210681e-02]
     [ -2.62861992e-06  -3.76977022e-01  -4.61457906e-01   2.19360606e-05
        1.17192702e-05   3.05190352e-02]
     [  5.23878935e-02  -4.05504085e-01  -4.13155692e-01   1.82846091e-05
        2.96514932e-06   3.73232706e-02]]
    [[  0.00000000e+00   4.57543600e-01   4.31569927e-01   5.19391799e-01
        5.94504488e-01   6.02742176e-01]
     [  0.00000000e+00   0.00000000e+00  -7.96265339e-06   6.01127948e-01
        6.15244114e-01   7.79807376e-01]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   3.95748322e-01
        5.31248242e-01   5.91957421e-01]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
        3.96141893e-01   5.19728674e-01]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
        0.00000000e+00   2.90218514e-01]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
        0.00000000e+00   0.00000000e+00]]
    Velocities:
    [[ -1.09850813e-06   5.00607884e-06   6.16730839e-06   5.62377152e-06
        5.73877050e-06   5.50217616e-06]
     [ -5.85341536e-07   1.50290986e-06  -1.72337390e-06   8.46918447e-07
        8.17447121e-07   1.74348716e-06]
     [ -1.44219558e-06   3.71645581e-06  -2.08920058e-06  -6.76512020e-07
        1.05287133e-06   2.11027366e-06]]
    [[  0.00000000e+00  -7.66207110e-07  -6.09268574e-06  -5.67125374e-06
       -5.76021402e-06  -5.57690770e-06]
     [  0.00000000e+00   0.00000000e+00   7.08526459e-07  -2.78360838e-06
       -4.44125870e-06  -1.02747299e-06]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00  -1.71464998e-06
       -3.95302432e-06   6.56650749e-07]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
       -4.17784696e-06  -1.18798834e-07]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
        0.00000000e+00   3.00263240e-06]
     [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
        0.00000000e+00   0.00000000e+00]]
    MSE = 9.02963482702e-05
    CPU times: user 4min 46s, sys: 1.11 s, total: 4min 47s
    Wall time: 4min 57s

#### Crossed diamond
In this concurrency structure two serial transactions are executed concurrently with another two serial transactions so that the barriers between the serial transactions are connected with a fifth transaction. This is a structure that cannot be composed using the serial and parallel composition operators.

In [198]:
%%time
class Diamond(object):
    
    # alpha=-100, beta=0.0 ensure the child response times do not contain
    # additive noise
    graph = FSBGraph()
    graph.add_edge('start', 'B1', alpha=-100, beta=0.0)
    graph.add_edge('start', 'B2', alpha=-100, beta=0.0)
    graph.add_edge('B1', 'end', alpha=-100, beta=0.0)
    graph.add_edge('B2', 'end', alpha=-100, beta=0.0)
    graph.add_edge('B1', 'B2', alpha=-100, beta=0.0)

    #graph.start_node, graph.end_node = graph.nodes['start'], graph.nodes['end']
    graph.start_node, graph.end_node = 'start', 'end'
    data = FSBSampler(graph, num_children=5)
    RNN = createNWayRNN(5, leak=-1.0, oversizing=5)
    RNN(step_size=1e-9, step_size_min=1e-12, momentum=0.99, l_min=-3, l_max=-1, 
        training_steps=20000, data=data, 
    batch_size=128, MSE_limit=1e-4, rng=np.random.RandomState(1236), num_runs=1)

Generating 0 training, 0 validation and 0 test samples using the following concurrency structure:
Training the model using the following parameters:
step_size = 1e-09,	lambda_min = -3,	lambda_max = -1,	training_steps = 20000

Batch #0: lambda=0.100, Cost=42.809, MSE=27.164, step_size=0.000
Weights:
[[ -4.84002329e-05  -2.67039202e-06   5.66143512e-05   8.44823334e-05
   -5.66540489e-05   1.64224910e-05  -2.69228365e-05  -4.75902384e-05
   -1.88779817e-05  -1.29415716e-05  -2.07131665e-05   9.47893930e-05
    6.15415076e-05  -5.78330522e-05  -5.82762155e-05  -5.92338170e-05
    1.91878123e-05   3.31474934e-05  -1.37858384e-05  -4.46747035e-05
    6.60788409e-05  -5.31524293e-06   1.86304547e-05  -1.95847813e-05
   -5.96385074e-05]
 [  3.82303195e-05   3.65078773e-05  -4.87963415e-05  -2.26552411e-05
   -8.89183582e-05   6.53427529e-05  -8.57089846e-05  -3.60831899e-06
   -1.38050081e-05   6.45472912e-05  -1.56925147e-05  -1.64612897e-05
    1.43841857e-05  -1.01884792e-05   5.31373082e-

    %%time
    class Diamond(object):

        # alpha=-100, beta=0.0 ensure the child response times do not contain
        # additive noise
        graph = FSBGraph()
        graph.add_edge('start', 'B1', alpha=-100, beta=0.0)
        graph.add_edge('start', 'B2', alpha=-100, beta=0.0)
        graph.add_edge('B1', 'end', alpha=-100, beta=0.0)
        graph.add_edge('B2', 'end', alpha=-100, beta=0.0)
        graph.add_edge('B1', 'B2', alpha=-100, beta=0.0)

        #graph.start_node, graph.end_node = graph.nodes['start'], graph.nodes['end']
        graph.start_node, graph.end_node = 'start', 'end'
        data = FSBSampler(graph, num_children=5)
        RNN = createNWayRNN(5, leak=-1.0, oversizing=3)
        RNN(step_size=1e-9, step_size_min=1e-12, momentum=0.99, l_min=-3, l_max=-1, 
            training_steps=20000, data=data, 
        batch_size=128, MSE_limit=1e-4, rng=np.random.RandomState(1236), num_runs=1)
    
        Generating 0 training, 0 validation and 0 test samples using the following concurrency structure:
    <__main__.FSBGraph object at 0x7fac7267ee90>
    Training the model using the following parameters:
    step_size = 1e-09,	lambda_min = -3,	lambda_max = -1,	training_steps = 20000

    Batch #0: lambda=0.100, Cost=31.044, MSE=25.990, step_size=0.000
    Weights:
    [[ -4.84002329e-05  -2.67039202e-06   5.66143512e-05   8.44823334e-05
       -5.66540489e-05   1.64224910e-05  -2.69228365e-05  -4.75902384e-05
       -1.88779817e-05  -1.29415716e-05  -2.07131665e-05   9.47893930e-05
        6.15415076e-05  -5.78330522e-05  -5.82762155e-05]
     [ -5.92338170e-05   1.91878123e-05   3.31474934e-05  -1.37858384e-05
       -4.46747035e-05   6.60788409e-05  -5.31524293e-06   1.86304547e-05
       -1.95847813e-05  -5.96385074e-05   3.82303195e-05   3.65078773e-05
       -4.87963415e-05  -2.26552411e-05  -8.89183582e-05]
     [  6.53427529e-05  -8.57089846e-05  -3.60831899e-06  -1.38050081e-05
        6.45472912e-05  -1.56925147e-05  -1.64612897e-05   1.43841857e-05
       -1.01884792e-05   5.31373082e-05   7.27960552e-05   3.11836834e-05
        8.65579851e-05   8.72822464e-05   3.09934208e-05]
     [  4.90193202e-06   7.40337086e-05   2.72390028e-05  -9.84251170e-05
        9.43031957e-05  -6.23039143e-05  -4.38500973e-05  -9.94962148e-06
        2.26507663e-05  -7.35048138e-05  -1.93402291e-05  -8.11892081e-06
        6.61248248e-05  -4.06783262e-05  -9.97866986e-05]
     [  1.47369883e-05   2.47858946e-05  -3.85503577e-05  -1.48588623e-05
        9.77451652e-05  -1.80956436e-05  -1.56577977e-05  -9.67146247e-05
       -2.59930213e-05  -7.70934902e-05  -1.13768323e-06   2.50007411e-05
        2.74317480e-05   9.36876124e-05  -7.61061078e-06]]
    [[ 0.         -0.45859002 -0.81787052  0.74232286 -0.16402436  0.55411482
      -0.89994653 -0.80900271 -0.41525209 -0.29754142 -0.81329708  0.92286086
      -0.71930555 -0.69559996  0.25214081]
     [ 0.          0.          0.62239093 -0.0873179  -0.65581638 -0.53799768
       0.29614305 -0.75033446  0.80336845 -0.27581908 -0.73281798  0.05527045
       0.12151516  0.73766107 -0.15017411]
     [ 0.          0.          0.          0.12271343  0.74951299  0.8739849
       0.44534753 -0.83146341  0.97736496 -0.46630029  0.5161862   0.79929512
      -0.32480223 -0.3328183   0.52996831]
     [ 0.          0.          0.          0.         -0.3093446   0.34488657
       0.86685365  0.26083188 -0.42698889  0.25423013  0.55807515  0.38804754
       0.14411054 -0.06613981  0.29228124]
     [ 0.          0.          0.          0.          0.          0.87259012
      -0.65337068 -0.68123198  0.22773375 -0.16306208 -0.84977846 -0.33697502
       0.35093956 -0.69996503  0.34247352]
     [ 0.          0.          0.          0.          0.          0.
      -0.62404737 -0.38903982 -0.830482   -0.21855661  0.96809018  0.52344266
      -0.46432166 -0.53446863  0.00447324]
     [ 0.          0.          0.          0.          0.          0.          0.
      -0.8885472   0.55824051 -0.8388439   0.01513049 -0.3108794  -0.64069582
       0.28975944  0.17003735]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.08692537  0.40678951  0.40665039 -0.39949121  0.1819933
       0.34254103 -0.22380521]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.         -0.23993579 -0.80249596 -0.16505775  0.23871507
       0.81060152  0.2780159 ]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.          0.          0.84762208  0.10258981 -0.28083043
       0.56404458 -0.3006999 ]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.          0.          0.         -0.03125927  0.78717328
       0.92627521 -0.75212091]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.          0.          0.          0.         -0.41438809
       0.23674352  0.04670038]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.          0.          0.          0.          0.
       0.0649499  -0.93911299]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.          0.          0.          0.          0.          0.
       0.94133066]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.          0.          0.          0.          0.          0.
       0.        ]]
    
    Batch #19999: lambda=0.001, Cost=24.985, MSE=24.934
    Weights:
    [[ -3.67927461e-08  -6.84333784e-08   1.05176066e-02  -5.63561709e-03
        4.13037733e-03   3.40989416e-03   6.92961195e-03  -4.96221389e-03
        2.13230478e-03  -1.68677575e-06   7.67830055e-03  -1.83706967e-03
        1.17740447e-02  -1.25965971e-02  -1.33839449e-02]
     [ -3.36008143e-08  -5.54280188e-08   9.59499214e-03  -5.23299531e-03
        3.82647668e-03   3.08850644e-03   6.30008515e-03  -4.48205753e-03
        2.02050359e-03  -9.80303379e-06   7.04724996e-03  -1.74225060e-03
        1.06204892e-02  -1.14402339e-02  -1.22232236e-02]
     [  4.59412331e-09  -1.55655591e-07   9.27718718e-03  -5.26872504e-03
        3.99072892e-03   2.81906811e-03   5.94470995e-03  -4.24853946e-03
        1.95895754e-03  -1.72690871e-06   6.78523966e-03  -1.60525573e-03
        1.02057603e-02  -1.06782610e-02  -1.14083461e-02]
     [ -2.70522684e-08  -1.03727090e-07   1.04756052e-02  -5.81702679e-03
        4.43594423e-03   3.17069702e-03   6.86858140e-03  -4.79548468e-03
        2.16593943e-03  -2.71071871e-05   7.50558623e-03  -1.82020749e-03
        1.15177944e-02  -1.23005708e-02  -1.31291123e-02]
     [ -3.28238555e-09   5.85298480e-09   1.00838873e-02  -5.81410060e-03
        4.45479057e-03   3.10775858e-03   6.59367419e-03  -4.80824988e-03
        2.28160061e-03  -6.19245503e-05   7.40060570e-03  -1.73679406e-03
        1.12793436e-02  -1.18767669e-02  -1.27219826e-02]]
    [[ 0.         -0.45855593 -0.81783631  0.74228873 -0.16399019  0.55408073
      -0.89991242 -0.80896867 -0.41521791 -0.29750732 -0.81326289  0.92282672
      -0.71927133 -0.69556596  0.25210659]
     [ 0.          0.          0.62235723 -0.08728397 -0.65578205 -0.53796344
       0.29610913 -0.75030057  0.80333444 -0.27578498 -0.73278356  0.05523621
       0.12148151  0.73762651 -0.15014049]
     [ 0.          0.          0.          0.12254412  0.74958228  0.8740273
       0.44547622 -0.8315417   0.97738148 -0.46626598  0.51632854  0.79921853
      -0.32450003 -0.33307115  0.52962924]
     [ 0.          0.          0.          0.         -0.3092647   0.3448863
       0.86689159  0.260748   -0.42693229  0.25419612  0.55811917  0.38799462
       0.14419514 -0.06623278  0.2921121 ]
     [ 0.          0.          0.          0.          0.          0.87263467
      -0.65316906 -0.68131342  0.22775171 -0.16302777 -0.8495629  -0.33698449
       0.35118115 -0.70022602  0.34212581]
     [ 0.          0.          0.          0.          0.          0.
      -0.62364859 -0.38925742 -0.83033442 -0.21852205  0.96845127  0.52331351
      -0.46368708 -0.53507726  0.00375608]
     [ 0.          0.          0.          0.          0.          0.          0.
      -0.8885795   0.55823631 -0.8388097   0.01520071 -0.31087027 -0.6405032
       0.28955572  0.16982298]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.08705816  0.40675604  0.40719767 -0.39959688  0.18284252
       0.34156151 -0.22477581]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.         -0.23990171 -0.80246173 -0.16502374  0.23868114
       0.8105673   0.27798167]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.          0.          0.84761857  0.10254837 -0.28074984
       0.56396076 -0.30071863]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.          0.          0.         -0.0314075   0.78829133
       0.92500784 -0.75339743]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.          0.          0.          0.         -0.41435372
       0.23670917  0.04666602]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.          0.          0.          0.          0.
       0.0638386  -0.94022364]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.          0.          0.          0.          0.          0.
       0.94064414]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.          0.          0.          0.          0.          0.
       0.        ]]
    MSE = 24.933584807
    CPU times: user 17min 10s, sys: 1.88 s, total: 17min 12s
    Wall time: 17min 34s    

**With uniform initialization there is no negative weight in $U$! **

    %%time
    class Diamond(object):

        # alpha=-100, beta=0.0 ensure the child response times do not contain
        # additive noise
        graph = FSBGraph()
        graph.add_edge('start', 'B1', alpha=-100, beta=0.0)
        graph.add_edge('start', 'B2', alpha=-100, beta=0.0)
        graph.add_edge('B1', 'end', alpha=-100, beta=0.0)
        graph.add_edge('B2', 'end', alpha=-100, beta=0.0)
        graph.add_edge('B1', 'B2', alpha=-100, beta=0.0)

        #graph.start_node, graph.end_node = graph.nodes['start'], graph.nodes['end']
        graph.start_node, graph.end_node = 'start', 'end'
        data = FSBSampler(graph, num_children=5)
        RNN = createNWayRNN(5, leak=-1.0, oversizing=2)
        RNN(step_size=1e-9, step_size_min=1e-12, momentum=0.99, l_min=-3, l_max=-1, 
            training_steps=20000, data=data, 
        batch_size=128, MSE_limit=1e-4, rng=np.random.RandomState(1236), num_runs=1)
        
    Generating 0 training, 0 validation and 0 test samples using the following concurrency structure:
    <__main__.FSBGraph object at 0x7fac79a76510>
    Training the model using the following parameters:
    step_size = 1e-09,	lambda_min = -3,	lambda_max = -1,	training_steps = 20000

    Batch #0: lambda=0.100, Cost=28.496, MSE=23.996, step_size=0.000
    Weights:
    [[ 0.0001  0.0001  0.0001  0.0001  0.0001  0.0001  0.0001  0.0001  0.0001
       0.0001]
     [ 0.0001  0.0001  0.0001  0.0001  0.0001  0.0001  0.0001  0.0001  0.0001
       0.0001]
     [ 0.0001  0.0001  0.0001  0.0001  0.0001  0.0001  0.0001  0.0001  0.0001
       0.0001]
     [ 0.0001  0.0001  0.0001  0.0001  0.0001  0.0001  0.0001  0.0001  0.0001
       0.0001]
     [ 0.0001  0.0001  0.0001  0.0001  0.0001  0.0001  0.0001  0.0001  0.0001
       0.0001]]
    [[ 0.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
     [ 0.  0.  1.  1.  1.  1.  1.  1.  1.  1.]
     [ 0.  0.  0.  1.  1.  1.  1.  1.  1.  1.]
     [ 0.  0.  0.  0.  1.  1.  1.  1.  1.  1.]
     [ 0.  0.  0.  0.  0.  1.  1.  1.  1.  1.]
     [ 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.]
     [ 0.  0.  0.  0.  0.  0.  0.  1.  1.  1.]
     [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.]
     [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
     [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
     

    Batch #19999: lambda=0.001, Cost=0.416, MSE=0.371
    Weights:
    [[  2.49457701e-03   1.28026967e-03   6.73093327e-04   3.69498763e-04
        2.17698899e-04   1.41797746e-04   1.03846556e-04   8.48706267e-05
        7.53824405e-05   7.53833245e-05]
     [  1.67557874e-03   8.70774877e-04   4.68347360e-04   2.67126401e-04
        1.66513016e-04   1.16204951e-04   9.10502274e-05   7.84724907e-05
        7.21833733e-05   7.21843647e-05]
     [  1.67428571e-03   8.70126326e-04   4.68022360e-04   2.66963576e-04
        1.66431446e-04   1.16164088e-04   9.10297587e-05   7.84622406e-05
        7.21782462e-05   7.21791869e-05]
     [  2.29053172e-03   1.17824705e-03   6.22081999e-04   3.43993085e-04
        2.04946053e-04   1.35421319e-04   1.00658340e-04   8.32765175e-05
        7.45853845e-05   7.45862713e-05]
     [  2.49385562e-03   1.27990590e-03   6.72910419e-04   3.69406863e-04
        2.17652735e-04   1.41774559e-04   1.03834912e-04   8.48647849e-05
        7.53795194e-05   7.53803243e-05]]
    [[ 0.          0.99986603  0.99991596  0.99994093  0.99995341  0.99995965
       0.99996277  0.99996433  0.99996511  0.99996511]
     [ 0.          0.          0.99989112  0.99992851  0.9999472   0.99995655
       0.99996122  0.99996356  0.99996473  0.99996473]
     [ 0.          0.          0.          0.9998974   0.99993165  0.99994877
       0.99995733  0.99996161  0.99996375  0.99996375]
     [ 0.          0.          0.          0.          0.99989897  0.99993243
       0.99994916  0.99995753  0.99996171  0.99996171]
     [ 0.          0.          0.          0.          0.          0.99989936
       0.99993263  0.99994926  0.99995758  0.99995758]
     [ 0.          0.          0.          0.          0.          0.
       0.99989946  0.99993268  0.99994928  0.99994928]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.99989948  0.99993269  0.99993269]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.99989949  0.99989949]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.          0.99983308]
     [ 0.          0.          0.          0.          0.          0.          0.
       0.          0.          0.        ]]
    MSE = 0.371415257934
    CPU times: user 15min 31s, sys: 2.23 s, total: 15min 34s
    Wall time: 15min 57s